In [1]:
#================================================================================
# 신규 ByteLevelBPETokenizer tokenizer vocab 생성 예제
# => tokenizer 방식중, ByteLevelBPETokenizer 를 이용하여 새롭게 tokenize vocab 을 생성하는 예제임(GPT에서는 SetnecePieceBPETokenzer 를 이용함)
# => 아래 과정은은 OpenAI GPT-2, ByteLevelBPETokenizer 를 사용함
# => 참고로 kogpt-2 방식으로, SetnecePieceBPETokenzer 를 사용함.
# 
# 참고자료 
# https://gist.github.com/lovit/e11c57877aae4286ade4c203d6c26a32
# https://discuss.huggingface.co/t/training-sentencepiece-from-scratch/3477/2
#
#================================================================================
# [과정] => gpt-2 tokenizer 방식
#
# 1. ByteLevelBPETokenizer 정의 후 훈련 
#     => 훈련시, corpora 목록, vocab_size, min_frequency 등을 설정함
#
# 2. 훈련한 ByteLevelBPETokenizer 저장
#     => stokenizer.save_model(OUT_PATH)
#     => 이대 저장되면, 해당 폴더에 2개의 파일이 생성됨(merges.txt, vocab.json)
#
# 3. GPT2TokenizerFast로 merges.txt, vocab.json 파일 불러옴
#     => transforer_tokenizer = GPT2TokenizerFast(vocab_file=vocab_path, merges_file=merges_path, add_prefix_space = True)
#
# 4. GPT2TokenizerFast tokenizer 저장
#     => transforer_tokenizer.save_pretrained(OUT_PATH) 
#     => 정상적으로 저장되면, 해당 폴더에 3개 json 파일이 생성됨(tokenizer.json, special_tokens_map.json, tokenizer_config.json)
#================================================================================

import torch
import numpy as np
import pandas as pd
import tokenizers
from tokenizers import (ByteLevelBPETokenizer,
                        CharBPETokenizer,
                        SentencePieceBPETokenizer,
                        BertWordPieceTokenizer)
# 말뭉치 지정
corpus_path = '../../korpora/kowiki_20190620/wiki_test.txt'


In [2]:
stokenizer = ByteLevelBPETokenizer(add_prefix_space = False, lowercase = False)

# 훈련
stokenizer.train(
    files = [corpus_path],
    vocab_size = 52000,  # 최대 vocab 계수 
    special_tokens = ["<cls>", "<eos>", "<mask>", "<unk>", "<pad>"],  # speical token 지정
    min_frequency = 5,   # 빈도수 
    show_progress = True,
    #limit_alphabet=10000, 
)

vocab = stokenizer.get_vocab()
print(f'vcoab 길이:{len(vocab)}')




vcoab 길이:710


In [3]:
# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = stokenizer.encode(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%stokenizer.decode(output.ids))

=>idx   : [172, 306, 287, 340, 375, 237, 251, 405, 311, 432, 113, 103, 292, 263, 106, 122, 362, 18]
=>tokens: ['ë', 'Ĥĺ', 'ëĬĶ', 'Ġìĺ', '¤ë', 'Ĭ', 'ĺ', 'ĠìķĦ', 'ì¹', '¨ë', '°', '¥', 'ìĿĦ', 'Ġë', '¨', '¹', 'ìĹĪëĭ¤', '.']
=>offset: [(0, 1), (0, 1), (1, 2), (2, 4), (3, 5), (4, 5), (4, 5), (5, 7), (7, 8), (7, 9), (8, 9), (8, 9), (9, 10), (10, 12), (11, 12), (11, 12), (12, 14), (14, 15)]
=>decode: 나는 오늘 아침밥을 먹었다.



In [4]:
import os
OUT_PATH = './mytokenbpe'
os.makedirs(OUT_PATH, exist_ok=True)
stokenizer.save_model(OUT_PATH)

['./mytokenbpe/vocab.json', './mytokenbpe/merges.txt']

In [5]:
# 3. GPT2TokenizerFast로 merges.txt, vocab.json 파일 불러옴
from transformers import GPT2TokenizerFast
vocab_path = './mytokenbpe/vocab.json'
merges_path = './mytokenbpe/merges.txt'

tokenizer = GPT2TokenizerFast(vocab_file=vocab_path, merges_file=merges_path)

In [6]:
# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = tokenizer.encode(sentence)
print(output)
decode = tokenizer.decode(output)
print(decode)

[172, 306, 287, 340, 375, 237, 251, 405, 311, 432, 113, 103, 292, 263, 106, 122, 362, 18]
나는 오늘 아침밥을 먹었다.


In [8]:
# PreTrainedTokenizerFast tokenizer 저장
import os
OUT_PATH = './mytoken_BPE'
os.makedirs(OUT_PATH, exist_ok=True)
tokenizer.save_pretrained(OUT_PATH)

('./mytoken_BPE/tokenizer_config.json',
 './mytoken_BPE/special_tokens_map.json',
 './mytoken_BPE/vocab.json',
 './mytoken_BPE/merges.txt',
 './mytoken_BPE/added_tokens.json',
 './mytoken_BPE/tokenizer.json')

In [9]:
from transformers import GPT2TokenizerFast
#OUT_PATH = 'gpt-2'
# 저장된 tokenzier 불러와서 확인
tokenizer = GPT2TokenizerFast.from_pretrained(OUT_PATH,
                                            bos_token='<cls>',
                                            eos_token='<eos>',
                                            unk_token='<unk>',
                                            pad_token='<pad>',
                                            mask_token='<mask>')

# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = tokenizer.encode(sentence)
print(output)
decode = tokenizer.decode(output)
print(decode)

[172, 306, 287, 340, 375, 237, 251, 405, 311, 432, 113, 103, 292, 263, 106, 122, 362, 18]
나는 오늘 아침밥을 먹었다.


In [ ]:
tokenizer.decode([23821, 246, 97, 167])